In [1]:
import numpy as np
import pandas as pd

# Análisis de la Data

El archivo cuenta con un total de 64 columnas. Sin embargo, del análisis de cada columna y la información necesaria para realizar el presente proyecto, sólo vamos a seleccionar 11. 

In [2]:
# Path para acceder al archivo csv
archivo = '../Data/Compras_modificado.csv'

# Selección de columnas
columnas_select = ['Coleccion', 'Referencia', 'Color', 'Descripcion', 'No.Pedido', 
                   'Tot_Piezas', 'Costo_Fabrica', 'Precio_Venta',
                   'Pais_Producto', 'Cta_Cliente', 'Nombre_Cliente', 
                   'Estatus', 'Plataforma', 'familia']

# Abrir archivo csv con columnas seleccionadas
data_df = pd.read_csv(archivo, usecols=columnas_select)

In [3]:
data_df.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia
0,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5590,12.0,15.65,25.0,CHINA,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK
1,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5595,12.0,15.65,25.0,CHINA,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK
2,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,CHINA,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK
3,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5599,4.0,15.65,25.0,CHINA,2305201,"MYL DE COLOMBIA,SAS",EN SISTEMA,ASIA,RK
4,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5600,4.0,15.65,25.0,CHINA,2305201,"MYL DE COLOMBIA,SAS",EN SISTEMA,ASIA,RK


## Análisis general de los datos 

In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65597 entries, 0 to 65596
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Coleccion       65597 non-null  object 
 1   Referencia      65597 non-null  object 
 2   Descripcion     65597 non-null  object 
 3   Color           65597 non-null  object 
 4   No.Pedido       65597 non-null  int64  
 5   Tot_Piezas      65597 non-null  float64
 6   Costo_Fabrica   65597 non-null  float64
 7   Precio_Venta    65597 non-null  float64
 8   Pais_Producto   65597 non-null  object 
 9   Cta_Cliente     65597 non-null  int64  
 10  Nombre_Cliente  65597 non-null  object 
 11  Estatus         65597 non-null  object 
 12  Plataforma      65597 non-null  object 
 13  familia         65597 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 7.0+ MB


In [5]:
data_df.describe()

,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Cta_Cliente
count,65597.000000,65597.000000,65597.000000,65597.000000,6.559700e+04
mean,7478.672805,17.126500,24.831679,44.497119,2.052870e+06
std,1367.369884,30.368732,12.421134,21.193926,4.587816e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,6405.000000,5.000000,17.470000,33.000000,2.201119e+06
50%,7228.000000,8.000000,26.600000,44.000000,2.224074e+06
75%,8747.000000,15.000000,31.910000,55.000000,2.306225e+06
max,9685.000000,849.000000,148.960000,1000.000000,2.601001e+06


In [6]:
data_df.isnull().sum()

Coleccion         0
Referencia        0
Descripcion       0
Color             0
No.Pedido         0
Tot_Piezas        0
Costo_Fabrica     0
Precio_Venta      0
Pais_Producto     0
Cta_Cliente       0
Nombre_Cliente    0
Estatus           0
Plataforma        0
familia           0
dtype: int64

**No hay datos nulos**

# Pre-procesamiento básico

En este pre-procesamiento básico, se van a realizar los siguientes ajustes:

1. Eliminar duplicados
2. Filtrar columna "Estatus" para eliminar pedidos "cancelados"
3. Filtrar clientes que desde 2019 no compran
4. Verificar existencia de datos nulos. Anteriormente se observó que no hay NaN, pero se verificarán otras formas de existencia de datos no válidos
4. Ajustar errores de texto en columnas con dato tipo obj

## Eliminar duplicados

In [7]:
data_df.shape

(65597, 14)

In [8]:
data_proces_df = data_df.drop_duplicates()

In [9]:
data_proces_df.shape

(65419, 14)

In [10]:
duplicados = data_df.shape[0] - data_proces_df.shape[0]
print(f'Se eliminaron {duplicados} filas duplicadas')

Se eliminaron 178 filas duplicadas


## Filtrado según estatus de la compra

Se eliminan las compras con estatus "cancelado" y "caida"

In [11]:
data_proces_df['Estatus'].unique()

array(['EN SISTEMA', 'CANCELADO', 'CAIDA', 'EN PROCESO',
       '               ', 'EN BODEGA', 'EN PUERTO', '03/22/2024',
       '02/16/2024', '02/23/2024', '02/14/2024', '02/19/2024',
       '03/08/2024', '02/09/2024'], dtype=object)

In [12]:
data_proces_df['Estatus'].value_counts()

EN SISTEMA         45387
CANCELADO          14164
                    4032
CAIDA                490
02/14/2024           452
03/22/2024           280
02/19/2024           182
02/16/2024           154
EN PROCESO           115
03/08/2024            70
EN PUERTO             38
02/23/2024            35
EN BODEGA             10
02/09/2024            10
Name: Estatus, dtype: int64

In [13]:
# Filtrar data set. Eliminamos compras canceladas o caídas
data_proces_status = data_proces_df[(data_proces_df['Estatus'] != 'CANCELADO') & (data_proces_df['Estatus'] != 'CAIDA')]

In [14]:
data_proces_status['Estatus'].unique()

array(['EN SISTEMA', 'EN PROCESO', '               ', 'EN BODEGA',
       'EN PUERTO', '03/22/2024', '02/16/2024', '02/23/2024',
       '02/14/2024', '02/19/2024', '03/08/2024', '02/09/2024'],
      dtype=object)

In [15]:
compras_filtradas = data_proces_status.shape[0]
compras_filtradas

50765

In [16]:
compras_canceladas = data_proces_df.shape[0] - compras_filtradas
print(f'Se filtró la base, se eliminaron {compras_canceladas} compras canceladas o caídas')

Se filtró la base, se eliminaron 14654 compras canceladas o caídas


## Datos nulos 

En el análisis inicial se observó que no hay datos NAN, pero se analizarán otras formas de datos nulos para como texto vacío, caracteres especiales, etc

In [17]:
columns = data_proces_status.columns
columns

Index(['Coleccion', 'Referencia', 'Descripcion', 'Color', 'No.Pedido',
       'Tot_Piezas', 'Costo_Fabrica', 'Precio_Venta', 'Pais_Producto',
       'Cta_Cliente', 'Nombre_Cliente', 'Estatus', 'Plataforma', 'familia'],
      dtype='object')

In [18]:
# Diccionario con valores únicos por columna
valores_unicos = {}

for column in columns:
    valores_u = data_proces_status[column].unique()
    valores_unicos[column] = valores_u

In [19]:
valores_unicos

{'Coleccion': array(['AW2016', 'AW2017', 'AW2018', 'AW2019', 'AW2020', 'AW2021',
        'FW2022', 'FW2023', 'FW2024', 'SS2016', 'SS2017', 'SS2018',
        'SS2019', 'SS2020', 'SS2021', 'SS2022', 'SS2023', 'SS2024'],
       dtype=object),
 'Referencia': array(['RK2321', 'RK2447', 'RK2464', ..., 'TJ7737 PE', 'TJ7950 PE',
        'TJ7951 PE'], dtype=object),
 'Descripcion': array(['GORRA LACOSTE PARA CABALLERO', 'MEDIAS LACOSTE PARA CABALLERO',
        'CAMISA PARA CABALLERO', 'JACKET PARA CABALLERO',
        'PANTALON LARGO PARA CABALLERO', 'CAMISA TIPO POLO PARA CABALLERO',
        'BERMUDA PARA CABALLERO', 'PANTALON CORTO PARA CABALLERO',
        'SUETER CON CAPUCHA PARA CABALLERO',
        'T-SHIRT CUELLO REDONDO PARA CABALLERO',
        'T-SHIRT CUELLO V PARA CABALLERO', 'T-SHIRT PARA CABALLERO',
        'SUETER MANGA LARGA CON CAPUCHA PARA CABALLERO',
        'CONJUNTO DEPORTIVO PARA CABALLERO', 'CARDIGAN PARA CABALLERO',
        'PULLOVER PARA CABALLERO', 'SUETER MANGA LARGA PARA

Se analizará columna por columna ya que se observan valores diferentes sólo por errores de escritura. También se reemplazarán los datos que son una cadena de texto vacío

## Filtrado de clientes que desde 2019 no realizan compras

El objetivo es filtrar la base de datos para obtener sólo los clientes activos a partir del 2020.

Pasos:

1. Modificar columna colección para unificar escritura. Se reemplaza AW por FW

2. Filtrar clientes que no compran desde 2019. Para esto:

    1. se filtran las colecciones FW y SS desde 2020
    2. calculo de los clientes activos (es decir que han realizado compras desde 2020)
    3. Filtrado de dataset total para mantener sólo clientes activos (se mantienen compras desde 2016 pero sólo de los clientes activos)

In [20]:
# función para reemplazar 'AW' por 'FW'
def standardize_coleccion_name(name):
    return name.replace('AW', 'FW')

# Aplicar la función a la columna 'Coleccion'
data_proces_status['Coleccion'] = data_proces_status['Coleccion'].apply(standardize_coleccion_name)

data_proces_status['Coleccion'].value_counts()

/tmp/ipykernel_1070/1125347918.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_proces_status['Coleccion'] = data_proces_status['Coleccion'].apply(standardize_coleccion_name)


FW2018    6277
FW2016    4096
SS2024    3860
SS2018    3550
FW2024    3433
SS2019    3414
SS2016    3323
FW2017    3115
FW2022    2911
SS2017    2752
SS2023    2344
FW2023    2337
FW2019    2133
SS2020    2076
FW2020    1891
SS2022    1591
FW2021    1041
SS2021     621
Name: Coleccion, dtype: int64

In [21]:
# Identificar las colecciones válidas desde SS2020 en adelante
colecciones_validas = data_proces_status['Coleccion'].unique()
colecciones_validas

array(['FW2016', 'FW2017', 'FW2018', 'FW2019', 'FW2020', 'FW2021',
       'FW2022', 'FW2023', 'FW2024', 'SS2016', 'SS2017', 'SS2018',
       'SS2019', 'SS2020', 'SS2021', 'SS2022', 'SS2023', 'SS2024'],
      dtype=object)

In [22]:
# Comprenhesion list donde se pasa a número los últimos dos caracteres de la columna Colección para filtrar las temporadas desde 2020
colecciones_validas = [col for col in colecciones_validas if int(col[4:]) >= 20]

In [23]:
colecciones_validas

['FW2020',
 'FW2021',
 'FW2022',
 'FW2023',
 'FW2024',
 'SS2020',
 'SS2021',
 'SS2022',
 'SS2023',
 'SS2024']

In [24]:
# máscara booleana para las filas que tienen colecciones válidas
mask = data_proces_status['Coleccion'].isin(colecciones_validas)

In [25]:
# DataFrame para obtener solo las filas con colecciones válidas
data_df_filtered = data_proces_status[mask]

In [26]:
data_df_filtered.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia
18907,FW2020,RF6002,MASCARILLA DE USO NO QUIRURGICO,031,1,100.0,3.85,7.0,PERU,2000006,RES.CLIENTES INTERNACIONALES,EN SISTEMA,,RF
18908,FW2020,RF6101,MASCARILLA DE USO NO QUIRURGICO,LAW,1,100.0,10.26,18.0,PERU,2000006,RES.CLIENTES INTERNACIONALES,EN SISTEMA,,RF
18909,FW2020,RF6103,MASCARILLA DE USO NO QUIRURGICO,LAW,1,100.0,10.26,18.0,FRANCIA,2000006,RES.CLIENTES INTERNACIONALES,EN SISTEMA,,RF
18910,FW2020,RF6123,MASCARILLA DE USO NO QUIRURGICO,2BE,1,41.0,8.11,14.0,FRANCIA,2000006,RES.CLIENTES INTERNACIONALES,EN SISTEMA,,RF
18921,FW2020,RK2321,GORRA PARA CABALLERO,031,7859,24.0,15.65,26.0,CHINA,1307082,"INVERSIONES GABYN,S.A.",EN SISTEMA,ASIA,RK


In [27]:
# Obtener una lista de clientes activos (que han comprado desde SS2019)
clientes_activos = data_df_filtered['Nombre_Cliente'].unique()

In [28]:
clientes_activos

array(['RES.CLIENTES INTERNACIONALES', 'INVERSIONES GABYN,S.A.',
       'RIPOSTO,S.A.', 'CELL STAR DA VINCI ENTERPRICES',
       'ALMACEN JERUSALEN', 'RES.SUCURSALES SR.JACK',
       'ICOSAL S.A. DE C.V.', 'ADN GROUP, S.A.',
       'INVERSIONES MONTIJO, S.A.', 'REMOTEXBO, S.A.',
       'DISTRIBUIDORA ATLANTIS', 'DISTRIBUIDORA RIMET HONDURAS, S.A.',
       'ICOSAL,S.A. AEROPUERT', 'OPTIKA, S.A.', 'CITY MALL (DAVID)',
       '                                        ', 'RES. GABYN S.A.',
       'CORP.DE FRANQUICIAS CENTRO AMERICANA,S.A',
       'INVERSIONES Y REPRES. SALAZAR 2015,C.A.', 'H&D COMPANY S.R.L',
       'TIENDA DICONS SRL (SANTIAGO)', 'RESERVA GABYN',
       'DISTRIBUIDORA 2224', 'JOSE N. BATARSE, S.A. DE C.V.',
       'RES. GABYN', 'DISTRIBUIDORA 2224, C.A.', 'PIU MEIS S.R.L.',
       'BENJAMIN ALEXIS MATUTE ESCOBAR-LAC.', 'TIENDA PALM',
       'CELL STAR DA VINCI ENTERPRISES',
       'YOUNG FASHION  RAMON HIPOLITO PEREZ',
       'RES. CLIENTES INTERNACIONALES', 'RES.GABYN(2)'

In [29]:
len(clientes_activos)

37

In [30]:
data_df_filtered['Coleccion'].value_counts()

SS2024    3860
FW2024    3433
FW2022    2911
SS2023    2344
FW2023    2337
SS2020    2076
FW2020    1891
SS2022    1591
FW2021    1041
SS2021     621
Name: Coleccion, dtype: int64

In [31]:
# Filtrar el DataFrame original para mantener solo los registros de clientes activos
data_df_final = data_proces_status[data_proces_status['Nombre_Cliente'].isin(clientes_activos)]

In [32]:
data_df_final['Coleccion'].value_counts()

SS2024    3860
FW2018    3560
FW2024    3433
FW2022    2911
SS2023    2344
FW2023    2337
SS2020    2076
FW2020    1891
FW2019    1826
SS2019    1621
SS2022    1591
SS2018    1408
FW2016    1349
FW2017    1097
FW2021    1041
SS2017     850
SS2016     822
SS2021     621
Name: Coleccion, dtype: int64

In [33]:
data_df_final.shape

(34638, 14)

In [34]:
compras_eliminadas = data_proces_status.shape[0] - data_df_final.shape[0]
compras_eliminadas

16127

In [35]:
# valores únicos de la columna 'Nombre_Cliente' en el DataFrame filtrado
clientes_restantes = data_df_final['Nombre_Cliente'].unique()

# Convertir a lista para mejor visualización 
lista_clientes_restantes = list(clientes_restantes)

lista_clientes_restantes

['RIPOSTO,S.A.',
 'ICOSAL S.A. DE C.V.',
 'ICOSAL,S.A. AEROPUERT',
 'INVERSIONES MONTIJO, S.A.',
 'REMOTEXBO, S.A.',
 'OPTIKA, S.A.',
 'RES.CLIENTES INTERNACIONALES',
 'ALMACEN JERUSALEN',
 'RES. CLIENTES INTERNACIONALES',
 'CORP.DE FRANQUICIAS CENTRO AMERICANA,S.A',
 'RES.GABYN',
 'RES.SUCURSALES SR.JACK',
 'DISTRIBUIDORA RIMET HONDURAS, S.A.',
 'DISTRIBUIDORA ATLANTIS',
 'CELL STAR DA VINCI ENTERPRICES',
 'CITY MALL (DAVID)',
 'INVERSIONES GABYN,S.A.',
 'ADN GROUP, S.A.',
 '                                        ',
 'RES. GABYN S.A.',
 'INVERSIONES Y REPRES. SALAZAR 2015,C.A.',
 'H&D COMPANY S.R.L',
 'TIENDA DICONS SRL (SANTIAGO)',
 'RESERVA GABYN',
 'DISTRIBUIDORA 2224',
 'JOSE N. BATARSE, S.A. DE C.V.',
 'RES. GABYN',
 'DISTRIBUIDORA 2224, C.A.',
 'PIU MEIS S.R.L.',
 'BENJAMIN ALEXIS MATUTE ESCOBAR-LAC.',
 'TIENDA PALM',
 'CELL STAR DA VINCI ENTERPRISES',
 'YOUNG FASHION  RAMON HIPOLITO PEREZ',
 'RES.GABYN(2)',
 'ANDRES GOMEZ FASHION STORE SRL',
 'TIENDA DICONS SRL (LA VEGA)',
 'P

In [36]:
len(lista_clientes_restantes)

37

## Análisis por columna. Ajustes

### Ajustes columnas "Pais Productor"

In [37]:
data_df_final['Pais_Producto'].unique()

array(['CHINA', 'RUMANIA', 'CHINA                    00042', 'VIETNAM',
       'INDONESIA', 'TUNISIA', 'PERU', 'EL SALVADO', 'MALASIA', 'FRANCIA',
       'MARRUECOS', 'COREA DEL SUR', 'LITUANIA', 'EL SALVADOR', 'TUNEZ',
       'BULGARIA', 'CAMBODIA', 'PORTUGAL', 'MAURICIO', 'TURQUIA',
       'SRI LANKA', 'COLOMBIA', 'GUATEMALA', 'TAILANDIA', 'MADAGASCAR',
       'CAMBOYA', 'EGIPTO', 'SERBIA', '                              ',
       'PARAGUAY', 'MEXICO', 'ITALIA', 'SERVIA'], dtype=object)

In [38]:
import string
import re

def basic_text_preprocess(sentence):
    """Función que realiza limpieza de texto. Incluye limpieza básica: eliminar espacios, pasar a minúscula,
    eliminar números y puntuación"""
    sentence = sentence.strip() #elimina espacios
    sentence = sentence.lower() #pasa texto a minúscula
    sentence = ''.join(char for char in sentence if not char.isdigit()) #elimina números

    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') #elimina puntuaciones
        
    # Eliminar espacios duplicados resultantes de las sustituciones
    sentence = re.sub(r'\s+', ' ', sentence).strip()
        
    return sentence

In [39]:
data_df_final.shape

(34638, 14)

In [40]:
data_df_final['Pais_Producto'] = data_df_final['Pais_Producto'].apply(basic_text_preprocess)

/tmp/ipykernel_1070/817777505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_final['Pais_Producto'] = data_df_final['Pais_Producto'].apply(basic_text_preprocess)


In [41]:
data_df_final.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia
2,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK
13,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5630,2.0,15.65,25.0,china,2224074,ICOSAL S.A. DE C.V.,EN SISTEMA,ASIA,RK
14,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5631,2.0,15.65,25.0,china,2224090,"ICOSAL,S.A. AEROPUERT",EN SISTEMA,ASIA,RK
15,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5635,5.0,15.65,25.0,china,2209051,"INVERSIONES MONTIJO, S.A.",EN SISTEMA,ASIA,RK
18,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,031,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK


In [42]:
data_df_final['Pais_Producto'].value_counts()

peru             13257
china             5348
turquia           2708
tunez             2226
vietnam           2150
sri lanka         1700
indonesia          990
colombia           870
cambodia           760
camboya            698
mauricio           678
marruecos          474
madagascar         459
rumania            392
francia            361
paraguay           253
guatemala          237
el salvador        228
tunisia            212
malasia            195
mexico              73
serbia              66
tailandia           65
egipto              47
el salvado          44
                    38
bulgaria            37
servia              30
lituania            24
portugal            14
corea del sur        2
italia               2
Name: Pais_Producto, dtype: int64

In [43]:
def replace_countries(sentence):
    '''Función para corregir errores de tipeo identificado en el análisis de valores únicos'''
    sentence = re.sub(r'\bcambodia\b', "camboya", sentence)
    sentence = re.sub(r'\bel salvado\b', "el salvador", sentence)
    sentence = re.sub(r'\btunisia\b', "tunez", sentence)
    sentence = re.sub(r'\bservia\b', "serbia", sentence)
    
    return sentence

In [44]:
# Aplico función para corregir errores de escritura en columna "Pais Productor"
data_df_final['Pais_Producto_fil'] = data_df_final['Pais_Producto'].apply(replace_countries)

/tmp/ipykernel_1070/579472978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_final['Pais_Producto_fil'] = data_df_final['Pais_Producto'].apply(replace_countries)


In [45]:
data_df_final['Pais_Producto_fil'].value_counts()

peru             13257
china             5348
turquia           2708
tunez             2438
vietnam           2150
sri lanka         1700
camboya           1458
indonesia          990
colombia           870
mauricio           678
marruecos          474
madagascar         459
rumania            392
francia            361
el salvador        272
paraguay           253
guatemala          237
malasia            195
serbia              96
mexico              73
tailandia           65
egipto              47
                    38
bulgaria            37
lituania            24
portugal            14
corea del sur        2
italia               2
Name: Pais_Producto_fil, dtype: int64

In [46]:
# Se observa que hay filas vacías que no son datos nulos, en este caso, se reemplaza por "sin dato"
data_df_final['Pais_Producto_fil'] = data_df_final['Pais_Producto_fil'].str.replace(r'^\s*$', 'sin dato', regex=True)

/tmp/ipykernel_1070/1286013631.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_final['Pais_Producto_fil'] = data_df_final['Pais_Producto_fil'].str.replace(r'^\s*$', 'sin dato', regex=True)


In [47]:
data_df_final['Pais_Producto_fil'].value_counts()

peru             13257
china             5348
turquia           2708
tunez             2438
vietnam           2150
sri lanka         1700
camboya           1458
indonesia          990
colombia           870
mauricio           678
marruecos          474
madagascar         459
rumania            392
francia            361
el salvador        272
paraguay           253
guatemala          237
malasia            195
serbia              96
mexico              73
tailandia           65
egipto              47
sin dato            38
bulgaria            37
lituania            24
portugal            14
corea del sur        2
italia               2
Name: Pais_Producto_fil, dtype: int64

### Ajustes columnas "Cliente"

Ajustes a realizar:
1. Analizar si cada cliente tiene asociado un número de cuenta único
2. Realizar ajustes en escritura
3. Resolver casos con datos en blanco (cadena de texto vacía)

In [48]:
data_df_final.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil
2,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china
13,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5630,2.0,15.65,25.0,china,2224074,ICOSAL S.A. DE C.V.,EN SISTEMA,ASIA,RK,china
14,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5631,2.0,15.65,25.0,china,2224090,"ICOSAL,S.A. AEROPUERT",EN SISTEMA,ASIA,RK,china
15,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5635,5.0,15.65,25.0,china,2209051,"INVERSIONES MONTIJO, S.A.",EN SISTEMA,ASIA,RK,china
18,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,031,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china


In [49]:
data_df_final.shape

(34638, 15)

In [50]:
len(data_df_final['Cta_Cliente'].unique())

29

En el dataset filtrado tenemos **37** nombres de clientes distintos y **29** números de cuentas distintos.

Calculamos la cantidad de nombres de clientes por número de Cuenta Cliente

In [63]:
# Calculo valores únicos de la columna 'Cta_Cliente' en el DataFrame filtrado final
cuenta_cliente = data_df_final['Cta_Cliente'].unique()

# Convertir a lista para mejor visualización 
lista_cuenta_cliente = list(cuenta_cliente)

lista_cuenta_cliente

[2313076,
 2224074,
 2224090,
 2209051,
 2306225,
 2601001,
 0,
 2000006,
 2220157,
 2218028,
 1307012,
 1207097,
 2214097,
 2214102,
 2403005,
 2220239,
 1307082,
 2327373,
 2401118,
 2401117,
 2327376,
 2224094,
 2201119,
 2214118,
 2218024,
 2401112,
 2401116,
 2401114,
 2327377]

In [71]:
len(data_df_final['Cta_Cliente'].unique())

29

In [72]:
# Loop para obtener los nombre de los clientes para cada cuenta cliente
clientes_unicos = {}

for num in cuenta_cliente:
    #Filtrado de dataset para obtener todas las filas que coinciden con el número de cuenta
    filas_cta_cliente = data_df_final[data_df_final['Cta_Cliente'] == num]
    
    #Obtenemos los nombres de clientes únicos 
    nombre_cliente = list(filas_cta_cliente['Nombre_Cliente'].unique())
   
    #Agregamos valor al diccionario donde key = cta cliente y valores es una lista de los nombre de clientes únicos
    clientes_unicos[num] = nombre_cliente

In [74]:
clientes_unicos

{2313076: ['RIPOSTO,S.A.'],
 2224074: ['ICOSAL S.A. DE C.V.'],
 2224090: ['ICOSAL,S.A. AEROPUERT'],
 2209051: ['INVERSIONES MONTIJO, S.A.'],
 2306225: ['REMOTEXBO, S.A.'],
 2601001: ['OPTIKA, S.A.'],
 0: ['RES.CLIENTES INTERNACIONALES',
  'RES. CLIENTES INTERNACIONALES',
  '                                        '],
 2000006: ['RES.CLIENTES INTERNACIONALES', 'RES. CLIENTES INTERNACIONALES'],
 2220157: ['ALMACEN JERUSALEN'],
 2218028: ['CORP.DE FRANQUICIAS CENTRO AMERICANA,S.A', 'ADN GROUP, S.A.'],
 1307012: ['RES.GABYN', 'RES. GABYN S.A.', 'RESERVA GABYN', 'RES. GABYN'],
 1207097: ['RES.SUCURSALES SR.JACK',
  'RES.GABYN(2)',
  '                                        '],
 2214097: ['DISTRIBUIDORA RIMET HONDURAS, S.A.'],
 2214102: ['DISTRIBUIDORA ATLANTIS'],
 2403005: ['CELL STAR DA VINCI ENTERPRICES', 'CELL STAR DA VINCI ENTERPRISES'],
 2220239: ['CITY MALL (DAVID)'],
 1307082: ['INVERSIONES GABYN,S.A.', 'RES.GABYN(2)'],
 2327373: ['INVERSIONES Y REPRES. SALAZAR 2015,C.A.'],
 2401118:

In [75]:
# Armamos diccionario con las cuentas clientes que tienen asociado más de 1 nombre cliente
clientes_con_varios_nombres = {}

for key,value in clientes_unicos.items():
    if len(value) > 1:
        nombres = list(value)
        clientes_con_varios_nombres[key] = nombres

In [76]:
clientes_con_varios_nombres

{0: ['RES.CLIENTES INTERNACIONALES',
  'RES. CLIENTES INTERNACIONALES',
  '                                        '],
 2000006: ['RES.CLIENTES INTERNACIONALES', 'RES. CLIENTES INTERNACIONALES'],
 2218028: ['CORP.DE FRANQUICIAS CENTRO AMERICANA,S.A', 'ADN GROUP, S.A.'],
 1307012: ['RES.GABYN', 'RES. GABYN S.A.', 'RESERVA GABYN', 'RES. GABYN'],
 1207097: ['RES.SUCURSALES SR.JACK',
  'RES.GABYN(2)',
  '                                        '],
 2403005: ['CELL STAR DA VINCI ENTERPRICES', 'CELL STAR DA VINCI ENTERPRISES'],
 1307082: ['INVERSIONES GABYN,S.A.', 'RES.GABYN(2)'],
 2327376: ['DISTRIBUIDORA 2224', 'DISTRIBUIDORA 2224, C.A.']}

En base al análisis anterior:
1. Valor "0" pasa a '2000006' -- res. clientes internacionales
2. El resto queda igual ya que son los mismos clientes, trabajaremos sobre **Cta. Cliente**

In [92]:
#función lambda para cambiar 0 por valor 2000006
data_df_final['Cta_Cliente_final'] = data_df_final['Cta_Cliente'].apply(lambda x: 2000006 if x == 0 else x)

/tmp/ipykernel_1070/2464154037.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_final['Cta_Cliente_final'] = data_df_final['Cta_Cliente'].apply(lambda x: 2000006 if x == 0 else x)


In [93]:
data_df_final.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil,Cta_Cliente_final
2,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china,2313076
13,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5630,2.0,15.65,25.0,china,2224074,ICOSAL S.A. DE C.V.,EN SISTEMA,ASIA,RK,china,2224074
14,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5631,2.0,15.65,25.0,china,2224090,"ICOSAL,S.A. AEROPUERT",EN SISTEMA,ASIA,RK,china,2224090
15,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5635,5.0,15.65,25.0,china,2209051,"INVERSIONES MONTIJO, S.A.",EN SISTEMA,ASIA,RK,china,2209051
18,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,031,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china,2313076


In [101]:
data_df_final['Cta_Cliente_final'].value_counts()

1207097    3608
2313076    3527
2224074    3291
2209051    3218
2000006    2335
1307012    2096
2306225    1960
2214102    1885
2220157    1834
2214097    1610
2218028    1511
2403005    1378
2224090    1193
2327376     847
2201119     825
2601001     758
2220239     606
2224094     587
1307082     529
2218024     330
2401117     222
2401112     149
2327373      84
2401116      78
2401118      69
2401114      51
2327377      38
2214118      19
Name: Cta_Cliente_final, dtype: int64

In [102]:
data_df_final.shape

(34638, 16)

### Ajustes columnas "Colección"

En este caso, vamos a realizar los siguientes pasos:
1. Para incorporar el tiempo, se reemplaza SS por may + año y FW por nov + año
2. Además hay que armar una columna sólo con SS y FW

In [105]:
# Crear una nueva columna sólo con SS o FW según corresponda
data_df_final['Temporada'] = data_df_final['Coleccion'].str.extract(r'(\w{2})')

/tmp/ipykernel_1070/914806934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_final['Temporada'] = data_df_final['Coleccion'].str.extract(r'(\w{2})')


In [106]:
data_df_final.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil,Cta_Cliente_final,Temporada
2,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china,2313076,FW
13,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5630,2.0,15.65,25.0,china,2224074,ICOSAL S.A. DE C.V.,EN SISTEMA,ASIA,RK,china,2224074,FW
14,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5631,2.0,15.65,25.0,china,2224090,"ICOSAL,S.A. AEROPUERT",EN SISTEMA,ASIA,RK,china,2224090,FW
15,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5635,5.0,15.65,25.0,china,2209051,"INVERSIONES MONTIJO, S.A.",EN SISTEMA,ASIA,RK,china,2209051,FW
18,FW2016,RK2321,GORRA LACOSTE PARA CABALLERO,031,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china,2313076,FW


Pasos:

1° duplicados, NaN, filtrar "cancelados", filtrar clientes que desde 2019 no compran

2° Colección: SS -- 01-05-2016   -   FW -- 01-11-2016
             Agregar 2 nueva columna SS o FW
   Referencia -- Familia -- "encoding" (agregar columnas)
   Total piezas (y - target)
   país productor -- corregir texto con distinta escritura (Servia, Camboya) 
                  -- encoding
   Cliente -- encoding
   Plataforma -- encoding

función para reemplazar 'AW' con 'FW'
def standardize_coleccion_name(name):
    return name.replace('AW', 'FW')

Aplicar la función a la columna 'Coleccion'
data_proces_status['Coleccion'] = data_proces_status['Coleccion'].apply(standardize_coleccion_name)

data_proces_status['Coleccion'].value_counts()